### Inferece NNI

In [1]:
import joblib

X_scaler = joblib.load('data/scalers/X_scaler.save')
y_scaler = joblib.load('data/scalers/y_scaler.save')

In [2]:
import pandas as pd 
expert_agent = pd.read_csv("data/data_generation_mpc_110_190_6_all.csv")

In [3]:
X = expert_agent[["inflow", "height"]]
y = expert_agent[["speed1_rpm", "speed4_rpm"]]

In [4]:
X_scaled = X_scaler.transform(X)
y_scaled = y_scaler.transform(y)

In [5]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim

class LitNeuralNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 2)
        self.train_loss = []
        self.val_loss = []

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        train_loss = nn.functional.mse_loss(y_hat, y)
        self.train_loss.append(train_loss.cpu().item())
        self.log('train_loss', train_loss, on_epoch=True, prog_bar=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.functional.mse_loss(y_hat, y)
        self.val_loss.append(val_loss.cpu().item())
        self.log('val_loss', val_loss, on_epoch=True, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        test_loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', test_loss, on_epoch=True)
        return test_loss

    def configure_optimizers(self):
        # Add L2 regularization with weight_decay
        optimizer = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

In [6]:
model = LitNeuralNet()

In [7]:
model = model.load_from_checkpoint("lightning_logs/41succeo/checkpoints/epoch=19-step=28900.ckpt")

In [8]:
from torch.utils.data import DataLoader, TensorDataset
import torch


torch_dataset = torch.tensor(X_scaled, dtype=torch.float32)

In [9]:
model.eval()

with torch.no_grad():
    # Assuming you have a tensor 'input_data'
    predictions = model(torch_dataset)


In [10]:
yhat = y_scaler.inverse_transform(predictions.cpu())

In [11]:
pump1_rpm_opt, pump4_rpm_opt = yhat[:, 0], yhat[:, 1]

#### Optional: Compare the performance of the NNI against the lookup table